In [1]:
import requests
import random
from bs4 import BeautifulSoup as soup
import ipywidgets as widgets
from IPython.display import display, clear_output
import webbrowser
from datetime import date

In [2]:
ratings = ['safe', 'explicit', 'questionable', 'any']
status_codes = {200: 'OK', 204: 'No Content', 400: 'Bad Request', 401: 'Unauthorized',
               403: 'Forbidden', 404: 'Not Found', 410: 'Gone', 420: 'Invalid Record',
               422: 'Locked', 423: 'Already Exist', 424: 'Invalid Parameters', 429: 'User Throttled',
               500: 'Internal Server Error', 502: 'Bad Gateway', 503: 'Service Unavailable'}

def space_to_underscore(string):
    return string.replace(' ', '_')

In [3]:
name = "Hu Tao"
full_name = "Hu Tao (Genshin Impact)"
day = 6
start_page = 1
end_page = 5
rating = 'safe'
no_rating = ''
source = 'https://safebooru.donmai.us'
max_width = 720
max_height = 360
today_date = date.today().strftime('%m/%d/%Y')
post_url = ""
img_url = ""
source_url = ""

In [4]:
# build query
tags = space_to_underscore(full_name)
if len(rating) != 0:
    tags += ' rating:{}'.format(rating)
if len(no_rating) != 0:
    tags += ' -rating:{}'.format(no_rating)
query = {'tags': tags}

In [5]:
def get_num_pages_images(source, query):
    num_pages = 0
    num_images = 0
    while True:
        query['page'] = num_pages + 1
        response = requests.get(source, params=query)
        articles = soup(response.text, 'html.parser').find_all('article')
        if len(articles) == 0:
            break
        num_pages += 1
        num_images += len(articles)
    return num_pages, num_images

def get_num_pages(source, query):
    return get_num_pages_images(source, query)[0]

def get_num_images(source, query):
    return get_num_pages_images(source, query)[1]

In [6]:
def get_post_ids(source, query, start_page, end_page):
    post_ids = []
    for page in range(start_page, end_page + 1):
        query['page'] = page
        response = requests.get(source, params=query)
        articles = soup(response.text, 'html.parser').find_all('article')
        for article in articles:
            post_ids.append(article['data-id'])
    return post_ids

In [7]:
post_ids = get_post_ids(source, query, start_page, end_page)

In [8]:
def get_random_post_id(post_ids):
    return random.choice(post_ids)

def get_random_post_url(post_ids):
    post_id = get_random_post_id(post_ids)
    return source + '/posts/' + post_id

def get_post_img_info(post_url):
    return soup(requests.get(post_url).text, 'html.parser').find(class_='image-container note-container')

def adjust_image_size(post_img_info):
    ori_width = int(post_img_info['data-width'])
    ori_height = int(post_img_info['data-height'])
    resize_ratio = min(max_width/ori_width, max_height/ori_height)
    disp_width = int(resize_ratio * ori_width)
    disp_height = int(resize_ratio * ori_height)
    return disp_width, disp_height

In [9]:
def get_image_widget(source, query, start_page, end_page):
    global post_url, img_url, source_url
    post_url = get_random_post_url(post_ids)
    post_img_info = get_post_img_info(post_url)
    source_url = post_img_info['data-normalized-source']
    disp_width, disp_height = adjust_image_size(post_img_info)
    img_url = post_img_info['data-file-url']
    img = widgets.HTML('<p style="text-align:center;"><img src="{}" width="{}" height="{}"></img></p>'
                   .format(img_url, disp_width, disp_height))
    return img

img = get_image_widget(source, query, start_page, end_page)

In [10]:
def get_header_widget():
    header = widgets.HTML('<h1 align="center">Daily {} posting #{}</h1> <h3 align="center"> ({}) </h3>'
                      .format(name, day, today_date))
    return header

header = get_header_widget()

In [11]:
def view_post(obj):
    webbrowser.open(post_url)
view_post_btn = widgets.Button(description = 'view post', layout=widgets.Layout(height='auto', width='auto'))
view_post_btn.style.button_color = 'lightblue'
view_post_btn.on_click(view_post)

def view_source(obj):
    webbrowser.open(source_url)
view_source_btn = widgets.Button(description = 'view source', layout=widgets.Layout(height='auto', width='auto'))
view_source_btn.style.button_color = 'lightgreen'
view_source_btn.on_click(view_source)

def download(obj):
    link = soup(requests.get(post_url).text, 'html.parser').find(id="post-option-download").find('a')['href']
    webbrowser.open(link)
download_btn = widgets.Button(description = 'download', layout=widgets.Layout(height='auto', width='auto'))
download_btn.style.button_color = 'orange'
download_btn.on_click(download)

def refresh(obj):
    out.clear_output(wait=True)
    global day
    day += 1
    header = get_header_widget()
    img = get_image_widget(source, query, start_page, end_page)
    with out:
        display(header, img, panel_box)
refresh_btn = widgets.Button(description = 'refresh', layout=widgets.Layout(height='auto', width='auto'))
refresh_btn.style.button_color = 'yellow'
refresh_btn.on_click(refresh)

In [12]:
panel = widgets.TwoByTwoLayout(top_left=view_post_btn,
               top_right=view_source_btn,
               bottom_left=download_btn,
               bottom_right=refresh_btn,
               layout=widgets.Layout(height='auto', width='500px'))
panel_layout = widgets.Layout(display='flex',
                flex_flow='column',
                align_items='center')
panel_box = widgets.HBox(children=[panel], layout=panel_layout)
out = widgets.Output()

In [13]:
with out:
    display(header, img, panel_box)

In [14]:
out

Output()